In [50]:
# Import libraries
import pandas as pd
import requests
import psycopg2
from bs4 import BeautifulSoup

In [51]:
def parser(url):
    
    # Plain HTML code
    plain = requests.get(url).text
    
    # Parser
    s = BeautifulSoup(plain, "html.parser")
    
    return s

In [58]:

conn = psycopg2.connect("dbname=coderschool user=lenovo password=1")
conn.autocommit = True
cur = conn.cursor()
cur.execute('SELECT c.cat_id AS Cc,c.url FROM categories AS c LEFT JOIN categories AS e ON e.parent_id = c.cat_id WHERE e.cat_id IS NULL;')
notes = cur.fetchall()


In [53]:
def get_connection():
    connection = psycopg2.connect(user = "lenovo",
                                  password = '1',
                                  host = "127.0.0.1",
                                  port = "5432",
                                  database = "coderschool")
    return connection

In [54]:
get_connection()

<connection object at 0x0000029596075268; dsn: 'user=lenovo password=xxx dbname=coderschool host=127.0.0.1 port=5432', closed: 0>

In [55]:
# Function to create an empty `products` table in our database if it doesn't exist yet
def create_products_table():
    query = (
        """ CREATE TABLE IF NOT EXISTS products2 (
                product_number SERIAL PRIMARY KEY,
                product_id INT,
                title TEXT,
                price INT,
                img_url TEXT,
                category_name TEXT,
                category_id INT
                );
    
        """)
    print('INFO create_products_table(): Create table products')
    
    try:
        # Create connection & cursor
        connection = get_connection()
        cursor = connection.cursor()
        
        # Execute the query
        cursor.execute(query)
        
        # Commit the changes made to our database
        connection.commit()

    except (Exception, psycopg2.Error) as error :
        print ("ERROR create_products_table(): Error while connecting to PostgreSQL", error)
        
        # In case of error, cancel all changes made to our database during the connection
        connection.close()
        return
    
    finally:
    
        # Close the connection & cursor
        cursor.close()
        connection.close()

In [47]:
create_products_table()


INFO create_products_table(): Create table products


In [56]:
class Products:
    def __init__(self, product_number, product_id, title, price, img_url, category_name, category_id):
        self.product_number = product_number
        self.product_id = product_id
        self.title = title
        self.price = price
        self.img_url = img_url
        self.category_name = category_name
        self.category_id = category_id
        
    def insert_data_to_db(self):
    
        print('INFO insert_data_to_db(): Insert data to DB')

        try:
            # Create connection & cursor 
            connection = get_connection()
            cursor = connection.cursor()

            # Create the query to insert product information to 'table_name'
            query =  f"""
                INSERT INTO products2 (product_id, title, price, img_url, category_name, category_id) 
                VALUES (%s, %s, %s, %s, %s, %s) RETURNING product_number;
            """
            val = (self.product_id, self.title, self.price, self.img_url, self.category_name, self.category_id)
            # Execute the query
            cursor.execute(query,val)

            # Commit the changes made to our database
            connection.commit()

        except (Exception, psycopg2.Error) as error :
            print ("ERROR save_data_to_db(): Error while connecting to PostgreSQL", error)

            # In case of error, cancel all changes made to our database during the connection
            connection.close()

        finally:

            # Close the connection & cursor
            cursor.close()
            connection.close()
            

def scrape(cat_id, url):

    # Initialize empty 'results' list
    results = []

    # Run Parser on the product page
    s = parser(url)

    # Find all tags <div class='product-item'> and store them in 'prodct_items' list, each tag represent a product
    product_items = s.findAll('div',{'class':'product-item'})

    # If the tag list is empty (i.e. the page doesn't have any product), return an empty list.
    if len(product_items) == 0:
        return []

    # If the tag list is not empty (i.e. the page has products),...
    else: 

        # Iterate through all product and store the product information in the 'row' list
        for i in range(len(product_items)):

            row = [product_items[i]['data-id'],            
                   product_items[i]['data-title'],
                   product_items[i]['data-price'], 
                   product_items[i].find('img',{'class':'product-image img-responsive'})['src'], 
                   product_items[i]['data-category'],
                   cat_id
                   ]   

            # Add the product information of each product into 'results' list
            results.append(row)

    # Return the list `results`   
    return results       

def scrape_all():
    print('INFO scrape_all(): Start scraping')

    # Create table `products` if it doesn't exist yet
    create_products_table()
    # Initialize 'queue' list with the results of get_urls()



    cur.execute(
    'SELECT c.cat_id AS Cc,c.url FROM categories AS c LEFT JOIN categories AS e ON e.parent_id = c.cat_id WHERE e.cat_id IS NULL;')
    notes = cur.fetchall()
    queue = notes
    # While there are items in `queue`,...
    while queue:
        temp = queue.pop()
        # `url` is set to the url of last item in `queue`
        url = temp[1]
        page = 1
        # `cat` is set to the category of last item in `queue`
        cat = temp[0]

        # Run scrape(cat, url) with given `cat` and `url`
        new_rows = scrape(cat,url)
        while new_rows:
            # If the result of scrape(cat, url) is not an empty list (i.e. the page has products),...  
            for item in new_rows:
                product = Products(None, item[0], item[1],item[2],item[3],item[4],item[5])
                # Insert the result of scrape(cat, url) to table `product`
                product.insert_data_to_db()

            # Generate next page url 
            page = page + 1
            url = url.split("&")[0]+"&page="+str(page)
            new_rows = scrape(cat,url)

        print('Add next page', page)

    print('Task completed!')

In [59]:
scrape_all()

INFO scrape_all(): Start scraping
INFO create_products_table(): Create table products
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert d

INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert

INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert

INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert

INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert data to DB
INFO insert_data_to_db(): Insert

KeyboardInterrupt: 

In [ ]:
def zii(a):
    url = a
    print(url)

In [ ]:
zii("success")

In [29]:
a = [1,2,3]
a.pop()

3

In [30]:
a


[1, 2]

In [ ]:
# Function to insert new data to our table
def insert_data_to_db(data):
    
    print('INFO insert_data_to_db(): Insert data to DB')
    
    try:
        # Create connection & cursor 
        connection = get_connection()
        cursor = connection.cursor()
        
        # For each row (product) in data (product page),...
        for row in data:
            # Create the query to insert product information to 'table_name'
            query =  f"""
            INSERT INTO products (product_id,title,price,img_url,category) 
            VALUES (%s, %s, %s, %s, %s) RETURNING product_number;
        """
            
            # Execute the query
            cursor.execute(query)
            
            # Commit the changes made to our database
            connection.commit()

    except (Exception, psycopg2.Error) as error :
        print ("ERROR save_data_to_db(): Error while connecting to PostgreSQL", error)
        
        # In case of error, cancel all changes made to our database during the connection
        connection.close()
        
    finally:
        
        # Close the connection & cursor
        cursor.close()
        connection.close()

In [ ]:
# Function to get all data from table 'products'
def select_data_from_db():
    
    try:
        # Create connection & cursor 
        connection = get_connection()
        cursor = connection.cursor()
        
        # Get all data from table 'products'
        query = 'SELECT * FROM products;'
        cursor.execute(query)      
        
        # Fetch (Collect) the data and save them in 'data'
        data = cursor.fetch()
        
        return data

    except (Exception, psycopg2.Error) as error :
        print ("ERROR save_data_to_db(): Error while connecting to PostgreSQL", error)
        
        # In case of error, cancel all changes made to our database during the connection
        connection.close()
        
    finally:
        
        # Close the connection & cursor
        cursor.close()
        connection.close()

In [ ]:
cat = get_urls()

In [ ]:
cat[-1][1]

In [ ]:
# Web scrape function
def scrape(cat, url):
    
    # Initialize empty 'results' list
    results = []
   
    # Run Parser on the product page
    s = parser(url)
    
    # Find all tags <div class='product-item'> and store them in 'prodct_items' list, each tag represent a product
    product_items = s.findAll('div',{'class':'product-item'})
    
    # If the tag list is empty (i.e. the page doesn't have any product), return an empty list.
    if len(product_items) == 0:
      return []

    # If the tag list is not empty (i.e. the page has products),...
    else: 
        
        # Iterate through all product and store the product information in the 'row' list
        for i in range(len(product_items)):

            row = [product_items[i]['data-id'],            
                   product_items[i]['data-title'],
                   product_items[i]['data-price'], 
                   product_items[i].find('img',{'class':'product-image img-responsive'})['src'], 
                   product_items[i]['data-category'],
                   cat]   

            # Add the product information of each product into 'results' list
            results.append(row)
            
    # Return the list `results`   
    return results

In [ ]:
a=scrape(cat[-1][0],cat[-1][1])

In [ ]:
data[0]

In [ ]:
# Function to insert new data to our table
def insert_data_to_db(data):
    
    print('INFO insert_data_to_db(): Insert data to DB')
    
    try:
        # Create connection & cursor 
        connection = get_connection()
        cursor = connection.cursor()
        
        # For each row (product) in data (product page),...
        for row in data:
            # Create the query to insert product information to 'table_name'
            query =  f"""
            INSERT INTO products (product_id, title, prices, img_url, category) 
            VALUES (%s, %s, %s, %s, %s) RETURNING product_number;
        """
            val = (row[0], row[1], row[2], row[3], row[4])
            
            
            # Execute the query
            cursor.execute(query, val)
            
            # Commit the changes made to our database
            connection.commit()

    except (Exception, psycopg2.Error) as error :
        print ("ERROR save_data_to_db(): Error while connecting to PostgreSQL", error)
        
        # In case of error, cancel all changes made to our database during the connection
        connection.close()
        
    finally:
        
        # Close the connection & cursor
        cursor.close()
        connection.close()

In [ ]:
insert_data_to_db(a)

In [ ]:
def scrape_all():
    print('INFO scrape_all(): Start scraping')
    
    # Initialize 'queue' list with the results of get_urls()
    queue = get_urls()
    
    # Create table `products` if it doesn't exist yet
    create_products_table()
    
    
    # While there are items in `queue`,...
    while queue is not None:
        for i in range (len(queue)):
          
          # `url` is set to the url of last item in `queue`
          url = queue[-1][1]
      
          # `cat` is set to the category of last item in `queue`
          cat = queue[-1][0]
    
          # Remove the last item in queue
          queue = queue[:-1]
          results = scrape(cat, url, cat_id)
          print('Scraping', cat)

          # Run scrape(cat, url) with given `cat` and `url`
          new_rows = scrape(cat,url)

          # If the result of scrape(cat, url) is not an empty list (i.e. the page has products),...  
          if new_rows is not None:
        
              # Insert the result of scrape(cat, url) to table `product`
              insert_data_to_db(new_rows, products)
        
              results += new_rows  # This is what we did in the first week
              
           
              # Generate next page url 
              page = page+1
              url= url.split("&")[0]+"&"+"page"+str(i+1)
        

            
              print('Add next page', page)
    
    print('Task completed!')

In [ ]:
scrape_all()

In [ ]:
l=[1,2,3,4,5,6,7,8]
for i in range (len(l)):
    print(l[-i -1])

In [ ]:
url = "https://tiki.vn/voucher-dich-vu/c11312?src=c.11312.hamburger_menu_fly_out_banner&page=1"
print(url.split("&")[0]+"&"+"page="+str(2))